In [1]:
import numpy as np
import pandas as pd 
import pandas_datareader as web
import random
import math
! pip3 install pygad
! pip3 install PyPortfolioOpt
# Input data files are available in the read-only "../input/" directory
import matplotlib.pyplot as plt
from pypfopt.expected_returns import mean_historical_return
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
import pygad
# Input data files are available in the read-only "../input/" directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### Returns

In [2]:
days = 1
d = {}
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if(filename!= "stock_metadata.csv" and filename!= "NIFTY50_all.csv"):
            df = pd.read_csv(os.path.join(dirname, filename))
            d[filename[:-4]] = df["Close"].pct_change(days)[0::days]

In [3]:
d

In [4]:
df1 = pd.DataFrame.from_dict(d, orient = 'index')
df = df1.dropna(axis=1, how='all')

In [5]:
df1 = pd.DataFrame.from_dict(d, orient = 'index')
df2 = df1.dropna(axis=1, how='all')
df2 = df2.fillna(0)
df2 = df2.T

In [6]:
df2.head()

In [7]:
def daily_results(portfolio):
    
    R = np.matmul(portfolio, Expected_Returns)
    print("Returns: ",100*R,"%")
    
    rportfolio = []
    p = len(multpl_stock_daily_returns)

    for i in range(p):
        ranreturns = multpl_stock_daily_returns.iloc[i]
        ranreturns[np.isnan(ranreturns)]=0
        y = np.matmul(portfolio, ranreturns)
        rportfolio.append(y)
    rportfolio.sort()
    
    #Standard Deviation
    stdev=np.std(rportfolio)
    print("Standard Deviation: ", stdev)

    #CVaR computation        
    kmin = (1-alpha)*p
    CVAR = 0
    for k in range(math.ceil(kmin)):
        CVAR += (1/p)*rportfolio[k]
    CVAR *= -1/(1-alpha)

    print("CVaR: ",CVAR)
    
    #Sharpe Ratio
    sharpe = (R-rf)/stdev
    print("Sharpe Ratio: ",sharpe)
    
    #Daily portfolio value
    portfolio_value= np.matmul(multpl_stocks['Adj Close'], portfolio)
    drawdown=(max(portfolio_value)-min(portfolio_value))/max(portfolio_value)
    
    #Calmar Ratio

    calmar=((1+R)**20-(1+rf)**20)/drawdown
    print("Calmar Ratio: ",calmar)
    
    return portfolio_value
    
    
    

In [8]:
df.head()

In [9]:
df['Expected Returns'] = df.mean(axis=1)

In [10]:
df["Expected Returns"]

In [11]:
stock_tickers= df.index.tolist()

In [12]:
stock_tickers_refined=[]
for x in stock_tickers:
    def res(x): return x+".NS"
    stock_tickers_refined.append(res(x))
    
for x in range(len(stock_tickers_refined)):
    if stock_tickers_refined[x]=="MM.NS":
        stock_tickers_refined[x]="M&M.NS"

In [13]:
multpl_stocks = web.get_data_yahoo(stock_tickers_refined,
start = "2021-04-30",
end = "2021-06-16")

In [14]:
multpl_stocks

In [15]:
multpl_stock_daily_returns = multpl_stocks['Adj Close'].pct_change(days)[0::days]
multpl_stock_daily_returns = multpl_stock_daily_returns.drop(labels="2021-04-30", axis=0)

In [16]:
Expected_Returns = multpl_stock_daily_returns.mean(axis=0)

In [17]:
S = 500

In [18]:
for i in range(S):
    df["S"+str(i+1)] = df.apply(lambda row: random.choice(row.tolist()), axis =1) 

In [19]:
df.head()

In [20]:
newdf = df.apply(lambda row : row.sample(n=S, replace=True).values, axis=1, result_type="expand")

In [21]:
newdf.head()

### GA

### Initialisation

In [22]:
pop = []
n = 100
m = 49
S = 500
k = 10000
w = 0.1

In [23]:
for i in range(n):
  x = []
  for j in range(m):
    x.append(random.uniform(0,1))
  x = np.array(x)
  x = x / sum(x)
  pop.append(x)

### Selection

In [24]:
def piminus(i):
#     print(i)
    if(i>=2):
        return wminus(d(i)) - wminus(d(i-1))
    else:
        return wminus(d(1))

In [25]:
def piplus(i):
    if(i<=S-1):
        return wplus(c(i)) - wplus(c(i+1))
    else:
        return wplus(c(S))

In [26]:
def d(i):
    return i/S

In [27]:
def c(i):
    return (S-i)/S

In [28]:
def wminus(di):
    return (di ** 0.69) / ((di ** 0.69) + ((1-di) ** 0.69)) ** (1/0.69)

In [29]:
def wplus(ci):
    return (ci ** 0.61) / ((ci ** 0.61) + ((1-ci) ** 0.61)) ** (1/0.61)

In [63]:
rf = 0.000199
a = 0.88
b = 0.88
lamda = 1
alpha = 0.95

In [64]:
def V(ret):
    if(ret>=0):
        return (ret ** a)
    else:
        return -lamda * ((-ret) ** b)

In [65]:
def fitness (portfolio, p_index):
    
    #print(portfolio)
    #portfolio sanity check
    for i in range(m):
        if(portfolio[i]<0):
            portfolio[i]=0
    portfolio = portfolio/sum(portfolio)

    rportfolio = []
    for i in range(S):
        ranreturns = newdf[i].to_numpy()
        ranreturns[np.isnan(ranreturns)]=0
        y = np.matmul(portfolio, ranreturns)
        rportfolio.append(y)
    rportfolio.sort()
    
    #prospect utility computation
    CPTU = 0;
    for j in range(S):
        if(rportfolio[j] < 0):
            CPTU += piminus(j+1)*V(rportfolio[j])
        else:
            CPTU += piplus(j+1)*V(rportfolio[j])
            
    #CVaR computation        
    kmin = (1-alpha)*S
    CVAR = 0
    for k in range(math.ceil(kmin)):
        CVAR += (1/S)*rportfolio[k]
    CVAR *= -1/(1-alpha)
    
    return CPTU / CVAR

In [33]:
def fitness2 (portfolio, p_index):
    
    #print(portfolio)
    #portfolio sanity check
    for i in range(m):
        if(portfolio[i]<0):
            portfolio[i]=0
    portfolio = portfolio/sum(portfolio)

    rportfolio = []
    for i in range(S):
        ranreturns = newdf[i].to_numpy()
        ranreturns[np.isnan(ranreturns)]=0
        y = np.matmul(portfolio, ranreturns)
        rportfolio.append(y)
    rportfolio.sort()
    
    #prospect utility computation
    CPTU = 0;
    for j in range(S):
        if(rportfolio[j]< 0):
            CPTU += piminus(j+1)*V(rportfolio[j])
        else:
            CPTU += piplus(j+1)*V(rportfolio[j])
            
    #CVaR computation        
    kmin = (1-alpha)*S
    CVAR = 0
    for k in range(math.ceil(kmin)):
        CVAR += (1/S)*rportfolio[k]
    CVAR *= -1/(1-alpha)
    
    return CPTU - k*max(CVAR-w,0)

In [67]:
x={
    'low':0,'high':1
}
h=10

In [69]:
num_generations = 300
num_parents_mating = int((h/100)*n)
initial_population = pop

fitness_function = fitness

num_genes = 49

init_range_low = 0
init_range_high = 1

gene_space=x

parent_selection_type = "sss"
keep_parents = 1

crossover_type = "single_point"

mutation_type = "random"
mutation_percent_genes = 10

ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       initial_population=initial_population,
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       gene_space=gene_space,
                       mutation_percent_genes=mutation_percent_genes)

In [70]:
ga_instance.run()

In [72]:
plt.xlabel("Generations")
plt.ylabel("Fitness Value")
plt.plot(ga_instance.best_solutions_fitness)
plt.savefig("graph.png")

In [73]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))

In [74]:
#Final Returns
for i in range(m):
        if(solution[i]<0):
            solution[i]=0
solution = solution/sum(solution)
# print(solution)
# print(solution.shape)
R = np.matmul(solution, Expected_Returns)
# print("Final Returns of portfolio: ", 100*R,"%")
# print("Annual Returns: ",((1+R)**250-1)*100,"%")

In [76]:
mu = mean_historical_return(df2, returns_data = True)
Shrink = CovarianceShrinkage(df2, returns_data = True).ledoit_wolf()

In [77]:
from pypfopt import EfficientFrontier
ef = EfficientFrontier(mu, Shrink)

In [78]:
A = ef.max_sharpe(risk_free_rate = 0.0002)
ef = EfficientFrontier(mu, Shrink)
B = ef.max_quadratic_utility()
ef = EfficientFrontier(mu, Shrink)
C = ef.efficient_return(target_return = 0.2)

In [79]:
max_sharpe = list(A.values()) 

In [80]:
max_quadratic_utility=list(B.values())

In [81]:
efficient_return=list(C.values())

In [82]:
#PUCRR
y = daily_results(solution)

In [83]:
x1 = [a / 25.41 for a in x]

In [58]:
y1 = [a / 46.09 for a in y]

In [53]:
for i in range (len(x1)):
    if i:
        x1[i] = x1[i]/1.01

In [ ]:
list(x.values)

In [85]:
plt.ylabel("Portfolio Value Index")
plt.xlabel("Day")
plt.plot(x1, label = "Naive")
plt.plot(y1, label = "PUCRR")
plt.legend()
plt.savefig("k.png")

In [ ]:
list(y.values)

In [91]:
z=daily_results(max_sharpe)

In [92]:
z

In [93]:
z1=[a / 88.88 for a in z]
z1

In [96]:
plt.ylabel("Portfolio Value Index")
plt.xlabel("Day")
plt.plot(z1, label = "Max Sharpe")
plt.plot(y1, label = "PUCRR")
plt.legend()
plt.savefig("sharpe.png")

In [87]:
daily_results(max_quadratic_utility)

In [88]:
daily_results(efficient_return)

In [89]:
#CVaR CONSTRAINT
daily_results(solution)

In [90]:
# Naive
naive =np.empty(m)
naive.fill(1/m)
x = daily_results(naive)